In [ ]:
import pathlib
import re

import rosbag
import geometry_msgs.msg as geomsg

import cameratransform as ct
from tf.transformations import quaternion_from_euler, euler_from_quaternion
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import numpy as np
import cv2

from cv_bridge import CvBridge

In [ ]:
!ls ..

In [ ]:
import postprocess

In [ ]:
bag = rosbag.Bag("../record.bag")
model_topic = "/gazebo/model_states"
camera_topic = "/robot/camera_rgb_00"
event_topic = "/robot/camera_dvs_00/events"
bridge = CvBridge()
g = postprocess.process_bag(bag, bridge, model_topic, camera_topic, event_topic)

In [ ]:
out = next(g)

In [ ]:
out

In [ ]:
models_path = pathlib.Path("../Models/")
dae_models = models_path.glob("*/*.dae")
meshes = {key.stem: postprocess.get_mesh(key) for key in dae_models}

In [ ]:
camera_poses = postprocess.camera_pose_generator(bag, model_topic)

In [ ]:
cameras = postprocess.camera_transform_generator(camera_poses, 0.5003983220157445 * 512)

In [ ]:
for (t, c) in camera:
    image = np.zeros()

In [ ]:
bagfile = "../record.bag"
bagfolder = pathlib.Path("../record.bag")
bag = rosbag.Bag(bagfile)

In [ ]:
model_topic = '/gazebo/model_states'
camera_topic = '/robot/camera_rgb_00'

In [ ]:
for topic, msg, t in bag.read_messages(topics=['/robot/camera_dvs_00/events']):
    print(dir(msg))
    coordinates = []
    for event in msg.events:
        coo = np.array([event.x, event.y, 1 if event.polarity else 0])
        coordinates.append(coo)
    break

In [ ]:
bridge = CvBridge()
count = 0
timestamp = 0
for topic, msg, t in bag.read_messages(topics=[camera_topic]):
    if count <37:
        count += 1
        continue
    timestamp = t
    image = bridge.imgmsg_to_cv2(msg, "bgr8")
    plt.imshow(image)
    msg.data = None
    break

In [ ]:
image.shape

In [ ]:
### count = 0
hammer_pose = None
for topic, msg, t in bag.read_messages(topics=[model_topic]):
    #if len(msg.pose) > 1:
    print(msg)
    if t >= timestamp and t.to_nsec() < 229716000000:
        hammer_pose = msg.pose[1]
        camera_pose = msg.pose[0]
        print(camera_pose)
        print(hammer_pose)
        break

In [ ]:
models_path = pathlib.Path('../Models')
hammer_file = models_path / 'hammer_simple' / 'hammer_simple.dae'

In [ ]:
with open(hammer_file) as fp:
    xml = fp.read()

In [ ]:
shape = "hammer_simple"

all_vertices_meshes = {shape: np.zeros((0, 3), dtype=float)}
all_triangles_meshes = {shape: np.zeros((0, 3), dtype=int)}
all_physical_scales = {shape: None}

vertices_info = re.findall(r'<float_array.+?mesh-positions-array.+?>(.+?)</float_array>', xml)
transform_info = re.findall(r'<matrix sid="transform">(.+?)</matrix>.+?<instance_geometry', xml, flags=re.DOTALL)  # better way?
triangles_info = re.findall(r'<triangles.+?<p>(.+?)</p>.+?</triangles>', xml, flags=re.DOTALL)
if len(triangles_info) == 0:
    triangles_info = re.findall(r'<polylist.+?<p>(.+?)</p>.+?</polylist>', xml, flags=re.DOTALL)
for part_id in range(len(vertices_info)):
    transform_matrix = np.array([float(n) for n in transform_info[part_id].split(' ')]).reshape((4, 4))
    vertices_temp = np.array([float(n) for n in vertices_info[part_id].split(' ')])
    vertices_temp = np.reshape(vertices_temp, (int(vertices_temp.shape[0]/3), 3))
    vertices_temp = np.dot(transform_matrix, np.c_[vertices_temp, np.ones(vertices_temp.shape[0])].T)[:-1].T
    triangles_temp = np.array([int(n) for n in triangles_info[part_id].split(' ')])[::3]
    triangles_temp = np.reshape(triangles_temp, (int(triangles_temp.shape[0]/3), 3))
    triangles_temp = triangles_temp + all_vertices_meshes[shape].shape[0]  # shift triangle indices
    all_vertices_meshes[shape] = np.vstack((all_vertices_meshes[shape], vertices_temp))
    all_triangles_meshes[shape] = np.vstack((all_triangles_meshes[shape], triangles_temp))
    

In [ ]:
orientation = hammer_pose.orientation
orientation_quat = np.array([orientation.x, orientation.y, orientation.z, orientation.w])
rotation = Rotation.from_quat(orientation_quat)
#rotation = Rotation.from_euler('z', hammer_pose.orientation.z)
position = np.array([hammer_pose.position.x, hammer_pose.position.y, hammer_pose.position.z])
shape_mesh = all_vertices_meshes[shape]
hammer_meshes = position + rotation.apply(shape_mesh)

hammer_meshes, position

In [ ]:
geomsg.Pose()

In [ ]:
def camera_from_position(camera_pose, resolution=(512, 512), focal_length=0.5003983220157445*512):
    orientation = camera_pose.orientation
    roll, pitch, yaw = euler_from_quaternion([orientation.x, orientation.y, orientation.z, orientation.w])
    (roll, pitch, yaw) = (angle_in_rad*180/np.pi for angle_in_rad in (roll, pitch, yaw))
    
    position = camera_pose.position
    camera_projection = ct.RectilinearProjection(focallength_px=focal_length, image=(512, 512))
    camera_orientation = ct.SpatialOrientation(pos_x_m=position.x, pos_y_m=position.y, elevation_m=position.z,
                    roll_deg=roll, tilt_deg=90-pitch, heading_deg=90-yaw)
    #camera_lens = ct.BrownLensDistortion(k1=0.0, k2=0.0, k3=0.0, projection=camera_projection)
    transformation = ct.Camera(projection=camera_projection, orientation=camera_orientation)#, lens=camera_lens)
    return transformation

dummy_pose = geomsg.Pose()
dummy_pose.position.x = 1
dummy_pose.position.y = 0.03
dummy_pose.position.z = 1
dummy_pose.orientation.z = -0.9773382675958592
dummy_pose.orientation.w = 0.21168351540147112
#camera = camera_from_position(dummy_pose)
#camera_pose.position.x = 0
camera_pose.position.y = 0.02
camera_pose.position.z = 1
camera = camera_from_position(camera_pose)
projected_mesh = camera.imageFromSpace(hammer_meshes)
triangle_mesh = np.take(projected_mesh, all_triangles_meshes['hammer_simple'], axis=0).astype(int)

distance_layer = np.zeros((512, 512, 3))
for mesh in triangle_mesh:
    cv2.fillConvexPoly(distance_layer, mesh, (1, 1, 1))
overlay = distance_layer + ( image / 300)
plt.imshow(overlay)
distance_layer.max()

In [ ]:
def compute_distances_to_cam(camera, vertices, distance_threshold = 0.1):
        camera_vector = np.array([camera.pos_x_m, camera.pos_y_m, camera.elevation_m])
        #cam_vector = np.array([l-p for (p,l) in zip(cam_pos, self.camera_look_at)])
        distances = vertices - camera_vector
        proj_dist = np.dot(camera_vector/np.linalg.norm(camera_vector), distances.T)
        norm_dist = np.linalg.norm(distances, axis=1)
        norm_dist[proj_dist < distance_threshold] = np.nan
        return norm_dist
distances = compute_distances_to_cam(camera, all_triangles_meshes['hammer_simple']).reshape(-1, 1).repeat(3, axis=1)
distances /= distances.max()
distances.shape

In [ ]:
mesh[0].astype(int)

In [ ]:
distance_layer = np.zeros((512, 512))

In [ ]:
np.array(list(map(lambda x: [x[0], -x[1]], triangle_mesh[0])))